In [4]:
import math


class Tracker:
    def __init__(self):
        # Store the center positions of the objects
        self.center_points = {}
        # Keep the count of the IDs
        # each time a new object id detected, the count will increase by one
        self.id_count = 0


    def update(self, objects_rect):
        # Objects boxes and ids
        objects_bbs_ids = []

        # Get center point of new object
        for rect in objects_rect:
            x, y, w, h = rect
            cx = (x + x + w) // 2
            cy = (y + y + h) // 2

            # Find out if that object was detected already
            same_object_detected = False
            for id, pt in self.center_points.items():
                dist = math.hypot(cx - pt[0], cy - pt[1])

                if dist < 35:
                    self.center_points[id] = (cx, cy)
#                    print(self.center_points)
                    objects_bbs_ids.append([x, y, w, h, id])
                    same_object_detected = True
                    break

            # New object is detected we assign the ID to that object
            if same_object_detected is False:
                self.center_points[self.id_count] = (cx, cy)
                objects_bbs_ids.append([x, y, w, h, self.id_count])
                self.id_count += 1

        # Clean the dictionary by center points to remove IDS not used anymore
        new_center_points = {}
        for obj_bb_id in objects_bbs_ids:
            _, _, _, _, object_id = obj_bb_id
            center = self.center_points[object_id]
            new_center_points[object_id] = center

        # Update dictionary with IDs not used removed
        self.center_points = new_center_points.copy()
        return objects_bbs_ids

In [13]:
import cv2
import pandas as pd
import numpy as np
from ultralytics import YOLO
import cvzone
from vidgear.gears import CamGear
from tracker import *

model = YOLO('yolov8s.pt')

# 다운로드한 동영상 파일 경로
video_path = 'sample.mp4'

# CamGear를 사용하여 동영상 스트리밍 시작
stream = CamGear(source=video_path).start()

def RGB(event, x, y, flags, param):
    if event == cv2.EVENT_MOUSEMOVE :  
        colorsBGR = [x, y]
        print(colorsBGR)
        

cv2.namedWindow('RGB')
cv2.setMouseCallback('RGB', RGB)

my_file = open("coco.txt", "r")
data = my_file.read()
class_list = data.split("\n")

count = 0
tracker = Tracker()

while True:    
    frame = stream.read()   
    count += 1
    if count % 2 != 0:
        continue

    frame = cv2.resize(frame, (1120, 720))
    results = model.predict(frame)
    a = results[0].boxes.data
    px = pd.DataFrame(a).astype("float")
    list = []
    for index, row in px.iterrows():
        x1 = int(row[0])
        y1 = int(row[1])
        x2 = int(row[2])
        y2 = int(row[3])
        d = int(row[5])
        c = class_list[d]
        if 'car' in c:
            list.append([x1, y1, x2, y2])
    bbox_idx = tracker.update(list)
    for bbox in bbox_idx:
        x3, y3, x4, y4, id1 = bbox
        cx = int(x3 + x4) // 2
        cy = int(y3 + y4) // 2
       
        cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)
        cv2.rectangle(frame, (x3, y3), (x4, y4), (255, 255, 255), 2) 
        cvzone.putTextRect(frame, f'{id1}', (x3, y3), 1, 1)

    cv2.imshow("RGB", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# CamGear 스트리밍 정리
stream.stop()
cv2.destroyAllWindows()


0: 416x640 8 cars, 1 umbrella, 154.9ms
Speed: 2.0ms preprocess, 154.9ms inference, 1.3ms postprocess per image at shape (1, 3, 416, 640)

0: 416x640 9 cars, 1 umbrella, 156.9ms
Speed: 2.4ms preprocess, 156.9ms inference, 1.0ms postprocess per image at shape (1, 3, 416, 640)



[712, 259]
[712, 259]


0: 416x640 8 cars, 1 umbrella, 155.3ms
Speed: 3.0ms preprocess, 155.3ms inference, 1.0ms postprocess per image at shape (1, 3, 416, 640)

0: 416x640 7 cars, 1 truck, 1 umbrella, 158.0ms
Speed: 2.5ms preprocess, 158.0ms inference, 1.0ms postprocess per image at shape (1, 3, 416, 640)

0: 416x640 7 cars, 1 truck, 1 umbrella, 159.9ms
Speed: 3.3ms preprocess, 159.9ms inference, 1.0ms postprocess per image at shape (1, 3, 416, 640)

0: 416x640 7 cars, 1 truck, 1 umbrella, 153.2ms
Speed: 3.7ms preprocess, 153.2ms inference, 1.0ms postprocess per image at shape (1, 3, 416, 640)

0: 416x640 8 cars, 1 umbrella, 183.1ms
Speed: 3.2ms preprocess, 183.1ms inference, 0.0ms postprocess per image at shape (1, 3, 416, 640)

0: 416x640 7 cars, 1 truck, 1 umbrella, 159.6ms
Speed: 3.0ms preprocess, 159.6ms inference, 1.0ms postprocess per image at shape (1, 3, 416, 640)

0: 416x640 8 cars, 1 umbrella, 150.9ms
Speed: 2.9ms preprocess, 150.9ms inference, 1.0ms postprocess per image at shape (1, 3, 416, 640)

[418, 40]
[418, 16]


0: 416x640 9 cars, 1 truck, 1 umbrella, 152.7ms
Speed: 5.1ms preprocess, 152.7ms inference, 1.1ms postprocess per image at shape (1, 3, 416, 640)

0: 416x640 9 cars, 1 umbrella, 152.2ms
Speed: 2.5ms preprocess, 152.2ms inference, 1.0ms postprocess per image at shape (1, 3, 416, 640)



...

0: 416x640 10 cars, 1 umbrella, 151.2ms
Speed: 4.5ms preprocess, 151.2ms inference, 1.3ms postprocess per image at shape (1, 3, 416, 640)

0: 416x640 10 cars, 1 umbrella, 165.2ms
Speed: 3.5ms preprocess, 165.2ms inference, 1.0ms postprocess per image at shape (1, 3, 416, 640)

0: 416x640 9 cars, 1 umbrella, 155.8ms
Speed: 3.0ms preprocess, 155.8ms inference, 0.0ms postprocess per image at shape (1, 3, 416, 640)



[730, 270]
[730, 269]
[712, 213]


0: 416x640 8 cars, 1 truck, 1 umbrella, 147.8ms
Speed: 3.1ms preprocess, 147.8ms inference, 1.0ms postprocess per image at shape (1, 3, 416, 640)

0: 416x640 8 cars, 1 umbrella, 175.8ms


[683, 194]


Speed: 2.5ms preprocess, 175.8ms inference, 1.0ms postprocess per image at shape (1, 3, 416, 640)


In [14]:
%pwd

'c:\\Users\\lenovo\\Desktop\\capstone\\vehicle_detection_tracker-main'